# Example: Compare Mean Fluorescence

This notebook demonstrates how to compare the mean fluorescence of a given channel between conditions in an experiment.

### Import dependencies

In [ ]:
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
from statsmodels.stats.multicomp import MultiComparison
pd.set_option('display.width', 2000)

### Read csv documents

In [ ]:
nucleus_df = pd.read_csv('nuclei.csv')
image_df = pd.read_csv('images.csv')


### Merge image and nucleus dataframes

Merging the dataframes on ImageName such that the background for the channel that we will measure is available for further background subtraction.

In [ ]:
merged_df = pd.merge(nucleus_df, image_df[['ImageName','Ch1Background']], on='ImageName')
print(merged_df.head())

### Subtract background

In [ ]:
mean_ch3_intensity = nucleus_df.groupby(['Condition', 'ImageName'])['Ch1Intensity'].mean().reset_index()
merged_ch3 = merged_df.groupby(['Condition','ImageName', 'Ch1Background'])['Ch1Intensity'].mean().reset_index()
merged_ch3['BackgroundSubtracted'] = merged_ch3['Ch1Intensity'] - merged_ch3['Ch1Background']

### Visualize results

In [ ]:
order =['Sham', 'Contra', 'Ipsi']

palette = sns.color_palette(['royalblue'], 2)
boxplot = sns.boxplot(data=merged_ch3, showfliers = False, x='Condition', y='BackgroundSubtracted', hue= 'Condition',order=order, palette='Set3')
stripplot = sns.stripplot(data=merged_ch3, x='Condition', y='BackgroundSubtracted', order=order, dodge=True, color='royalblue', alpha=0.7)
handles, labels = boxplot.get_legend_handles_labels()

plt.ylabel('Channel 1 fluroescence')

plt.show()

### Statistical analysis

A one way ANOVA is conducted followed by post-hoc testing.

In [ ]:
from scipy.stats import f_oneway
from statsmodels.stats.multicomp import pairwise_tukeyhsd


# Perform ANOVA test on filtered data
anova_results = f_oneway(merged_ch3[merged_ch3['Condition'] == 'Contra']['BackgroundSubtracted'],
                         merged_ch3[merged_ch3['Condition'] == 'Ipsi']['BackgroundSubtracted'],
                         merged_ch3[merged_ch3['Condition'] == 'Sham']['BackgroundSubtracted'])

# Perform Tukey HSD post hoc test
tukey_results = pairwise_tukeyhsd(merged_ch3['BackgroundSubtracted'], merged_ch3['Condition'])

print("ANOVA Results:")
print("F-statistic:", anova_results.statistic)
print("p-value:", anova_results.pvalue)

print("\nTukey HSD Results:")
print(tukey_results)